<a href="https://colab.research.google.com/github/ajay-sainy/notebooks/blob/main/audio_cloning_data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydub
!pip install --upgrade yt-dlp

In [ ]:
#@title Youtube Downloader - Simple Mode { vertical-output: true }
subType = "srv1"
subName = "sub.en." + subType
audioPath = "audio.mp3"
url = "https://www.youtube.com/watch?v=Nb5cBkbQpGY" #@param {type:"string"}
!yt-dlp --write-auto-subs --sub-format {subType} --skip-download {url} -o "subtitle:sub"
#!yt-dlp --list-subs $url
!yt-dlp -f 'ba' -x --audio-format mp3 {url} -o 'audio.%(ext)s'

In [ ]:
import xml.etree.ElementTree as ET
import html
import os

tree = ET.parse(subName)
root = tree.getroot()

# &amp;amp; -> & -> and
# &amp;#39; -> you&amp;#39;ll -> you'll -> you will
result = []
for line in root.iter('text'):
    # TODO: skip if contains music etc []
    text = html.unescape(line.text).strip() # TODO: Take care of vocab/Dictonary. Maybe remove/replace "&" and "'"
    start = float(line.attrib["start"])
    #print(start, " -> ", text)
    result.append((start, text))

finalResult = [] # start, end, length, text
for i in range(len(result) - 1):
  # print(item)

  item = result[i]
  nextItem = result[i + 1]

  start = item[0]
  text = item[1]
  length = round(nextItem[0] - start, 3)
  end = nextItem[0]
  if "[" in text:
    print("Skipping", text, item)
  else:
    finalResult.append((start, end, length, text))
print(finalResult[0:5])

In [ ]:
from pydub import AudioSegment
chunksInfo = finalResult

audio = AudioSegment.from_mp3(audioPath)

In [ ]:
from pydub.silence import split_on_silence
# Remove silences in the clips
def removeSilences(inputAudio, output_file_path, audio_format):
  audio_chunks = split_on_silence(inputAudio
                              ,min_silence_len = 100
                              ,silence_thresh = -45
                              ,keep_silence = 50
                          )

  # Putting the file back together
  combined = AudioSegment.empty()
  for chunk in audio_chunks:
      combined += chunk
  combined.export(output_file_path, format=audio_format)
  print("Removed silence (sec):", inputAudio.duration_seconds - combined.duration_seconds)

def ensure_dir(path):
  os.makedirs(path, exist_ok=True)

In [ ]:
final_output_folder = "final"
# create a directory to store the audio chunks
ensure_dir(final_output_folder)

exportFormat = "mp3"
for i, chunkInfo in enumerate(chunksInfo[0:10]):
  print(chunkInfo)
  start = chunkInfo[0] * 1000 #Works in milliseconds
  end = chunkInfo[1] * 1000
  chunk = audio[start:end]
  chunkName = str(i) + '.' + exportFormat
  chunkPath = final_output_folder + "/" + chunkName
  #removeSilences(chunk, chunkPath, exportFormat)
  chunk.export(chunkPath, format=exportFormat)
# Trim silence - https://stackoverflow.com/questions/29547218/remove-silence-at-the-beginning-and-at-the-end-of-wave-files-with-pydub

In [ ]:
# chop into clips with transcript
# trim silences
# remove noises, background sound, music etc - spleeter ?
# trim silences ??
# Done!!!
# https://manpages.ubuntu.com/manpages/kinetic/en/man1/mp3splt.1.html
# https://askubuntu.com/a/27637


In [ ]:
!rm final/ -r
!rm audio_chunks/ -r